In [1]:
import os
import sys

# Get the absolute path to your local repository
repo_root = '/Users/vivi/Desktop/eye_ai/repos/eye-ai-ml'  # Point to the specific repository


if repo_root not in sys.path:
    sys.path.insert(0, repo_root)

# Now you can import from your local repository
from eye_ai.eye_ai import EyeAI

In [1]:
# Prerequisites
import json
import os
from eye_ai.eye_ai import EyeAI

import pandas as pd
from pathlib import Path, PurePath
import logging

from deriva_ml import DatasetSpec, ExecutionConfiguration, DerivaML, Workflow, VersionPart
from deriva_ml import MLVocab as vc
from deriva_ml.deriva_definitions import ColumnDefinition, BuiltinTypes
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', force=True)

In [ ]:
# Login
from deriva.core.utils.globus_auth_utils import GlobusNativeLogin
host = 'dev.eye-ai.org'
# host = 'www.eye-ai.org'
catalog_id = "eye-ai"

gnl = GlobusNativeLogin(host=host)
if gnl.is_logged_in([host]):
    print("You are already logged in.")
else:
    gnl.login([host], no_local_server=True, no_browser=True, refresh_tokens=True, update_bdbag_keychain=True)
    print("Login Successful")

In [3]:
# cache_dir = '/data'
# working_dir = '/data'
cache_dir = '/Users/vivi/Desktop/eye_ai/execution_cache'
working_dir = '/Users/vivi/Desktop/eye_ai/execution_cache'
EA = EyeAI(hostname = host, catalog_id = catalog_id, cache_dir= cache_dir, working_dir=working_dir)

# Configuration

In [ ]:
# RID of source dataset, if any.
source_dataset = '2-7P5P' #'2-C9PR' '2-7P5P' '2-AGAW'

# workflow_instance = EA.create_workflow(
#     name="Multimodal template",
#     workflow_type="Test Workflow",
# )

workflow_instance = Workflow(
    name="Create Condition_Label feature",
    workflow_type="Feature_Creation",
    url='https://github.com/informatics-isi-edu/eye-ai-exec/blob/main/notebooks/templates/template_multimodal.ipynb',
    is_notebook=True
)

config = ExecutionConfiguration(
    datasets=[DatasetSpec(rid=source_dataset, version=EA.dataset_version(source_dataset), materialize=False)],
    assets=[],
    workflow= workflow_instance, # dev'5-SG9W'
    description="Template instance of a multimodal workflow")

# Initialize execution
execution = EA.create_execution(config)


In [ ]:
print(execution)

# Create DatasetBag

In [6]:
ds_bag = execution.datasets[0]

# Methods for multimodal data

### Extract modality

In [ ]:
modalities = EA.extract_modality(ds_bag)

In [ ]:
clinic = modalities['Clinic']
clinic


In [ ]:
len(ds_bag.get_table_as_dataframe('Subject')) # 1180
# ds_bag.get_table_as_dataframe('OCR_HVF')

### Retrive multimodal wide table

In [ ]:
wide = EA.multimodal_wide(ds_bag)

In [ ]:
print(len(wide))
wide.columns

# Upload results

In [ ]:
# crete asset path
asset_path = execution.asset_file_path("Execution_Asset", "wide_with_severity.csv")
wide.to_csv(asset_path)

# upload assets to catalog
execution.upload_execution_outputs(clean_folder=True)